In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# load data
df = pd.read_csv('employee_retention_data.csv')

In [ ]:
# explore data
print('shape: ',df.shape)
df.head()

In [ ]:
# seems like we could engineer a feature to see time at company. 

In [ ]:
# define quitters
df.loc[:, 'quit'] = 1
df.loc[df['quit_date'].isnull(), 'quit'] = 0

In [ ]:
# convert dates columns to datetime
df['join_date'] = pd.to_datetime(df['join_date'])
df['quit_date'] = pd.to_datetime(df['quit_date'])

In [ ]:
# engineer 'time at co' feature - calc timedelta and convert to float
def f(row):
    if row['quit'] == 1:
        val = ((row['quit_date'] - row['join_date'])/ np.timedelta64(1, 'Y'))
    elif row['quit'] == 0:
        val = ((pd.to_datetime('2015-12-13') - row['join_date'])/ np.timedelta64(1, 'Y'))
    return val

df['time_at_Co'] = df.apply(f, axis=1)

In [ ]:
df.seniority.hist()

In [ ]:
df.salary.hist()

In [ ]:
df.time_at_Co.hist()

In [ ]:
# let's look at time at company
sns.set(context = 'poster', style = 'white')
plt.figure(figsize=(20,10))
sns.violinplot(x = 'dept', y = 'time_at_Co',  hue = 'quit', data = df, split=True)

In [ ]:
# looks like a lot of people leave at 12 months and 24 months. 

In [ ]:
# look at salaries
#plt.figure(figsize=(20,10))
#sns.violinplot(x = 'dept', y = 'salary',  hue = 'quit', data = df, split=True)

In [ ]:
# might play some part, but not as stark as time_at_Co

In [ ]:
# look at experience
#plt.figure(figsize=(20,10))
#sns.violinplot(x = 'dept', y = 'seniority',  hue = 'quit', data = df, split=True)

In [ ]:
# looks like some outliers in marketing and engineering
# might need to tweak these features to bins instead of continuous

In [ ]:
# bin seniority into 3 bins = idea being junior, mid, and senior
#df['seniority_bin'] = pd.qcut(df.seniority, 3, labels=False)

In [ ]:
# looks to some slight differences between more experienced individuals and salary, but not at junior level

In [ ]:
# look at salary per experience amoung data scientists
#plt.figure(figsize=(20,10))
#sns.violinplot(x = 'seniority_bin', y = 'salary',  hue = 'quit', data = df.loc[df['dept'] == 'data_science',] , split=True)

In [ ]:
# look at salary per experience amoung data scientists
#plt.figure(figsize=(20,10))
#sns.violinplot(x = 'seniority_bin', y = 'time_at_Co',  hue = 'quit', data = df.loc[df['dept'] == 'data_science',] , split=True)

In [ ]:
# regardless of experience, bulk of quitters leave at 1 year, with spikes at 2 and 3 years
# my intutition is that this is enough time for individuals to feel the company out for fit, role, team, etc.

In [ ]:
# to make this model better, need to engineer features to distinguish employees who stay greater than 1 year. 

In [ ]:
# engineer feature to label salary with respect to peers
# calculate median salary per dept per seniority -
median_salary = df.groupby(['dept', 'seniority'])[['salary']].apply(np.median)
median_salary.name = 'median'
df = df.join(median_salary, on=['dept', 'seniority'])

# calculate the difference in average salary of individual to median
df['salary_diff'] = df['salary'] - df['median']
df.head()

In [ ]:
# look at salary_diff per dept
plt.figure(figsize=(20,10))
sns.violinplot(x = 'dept', y = 'salary_diff',  hue = 'quit', data = df , split=True)

In [ ]:
# this seems to indicate that relative salary isn't a major factor

In [ ]:
# look at salary_diff per company_id
plt.figure(figsize=(20,10))
sns.violinplot(x = 'company_id', y = 'salary_diff',  hue = 'quit', data = df , split=True)

In [ ]:
# company 11 looks weird. 
# 1 and 2 definitely pay the best
# not a visible difference between quitters and stayers with respect to salary diff, except at 11.

In [ ]:
df.head()

In [ ]:
# drop employee id, not important
df.drop(['employee_id', 'join_date', 'quit_date', 'median', 'salary_diff'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
# label encode dept
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['dept'] = le.fit_transform(df['dept'])

In [ ]:
df.head()

In [ ]:
# make sure there's no missing values
df.info()

In [ ]:
# split into training and test data
from sklearn.model_selection import train_test_split

y = df['quit'].values
X = df.drop(['quit'],axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
# Create a based model
lr = LogisticRegression()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = lr, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
from sklearn import metrics
lr = LogisticRegression(C=0.01)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
# Print the prediction accuracy
print (metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.svm import LinearSVC
clf = LinearSVC(dual=False, random_state=42)

param_grid = [{'C': [0.1, 1, 10, 100, 1000]}]

grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
from sklearn import metrics
# fit best model
clf = LinearSVC(C = 0.1, random_state = 42, dual=False)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
# Print the prediction accuracy
print (metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier(random_state = 42)

param_grid = {"n_estimators": [10, 20, 50, 100],
              "max_depth": [2, 3, 4, None],
              "max_features": [1, 2, 3, 'auto'],
              "min_samples_split": [2, 5, 10, 20],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
# fit best model
rf = RandomForestClassifier(n_estimators = 50,
                            max_depth = None,
                            max_features = 3,
                            min_samples_split = 20,
                            bootstrap = False,
                            criterion = 'entropy',
                            random_state = 42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
# Print the prediction accuracy
print (metrics.accuracy_score(y_test, y_pred))

In [ ]:
# optional
!pip install pdpbox

In [ ]:
from pdpbox import pdp, get_dataset, info_plots

# Create the data that we will plot
pdp_goals = pdp.pdp_isolate(model=rf, dataset=X_test, model_features=X_test.columns.tolist(), feature='time_at_Co')

# plot it
pdp.pdp_plot(pdp_goals, 'time_at_Co')
plt.show()

In [ ]:
# Create the data that we will plot
pdp_goals = pdp.pdp_isolate(model=rf, dataset=X_test, model_features=X_test.columns.tolist(), feature='salary')

# plot it
pdp.pdp_plot(pdp_goals, 'salary_diff')
plt.show()

In [ ]:
features_to_plot = ['time_at_Co', 'salary_diff']
inter1  =  pdp.pdp_interact(model=rf, dataset=X_test, model_features=X_test.columns.tolist(), features=features_to_plot)

pdp.pdp_interact_plot(pdp_interact_out=inter1, feature_names=features_to_plot, plot_type='contour')
plt.show()

In [ ]:
# optional
!pip install eli5

In [ ]:
# get feature importances using permutation 
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rf, random_state=42).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

In [ ]:
# optional
!pip install shap

In [ ]:
# visualize feature importances using SHAP
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(rf)

# calculate shap values. This is what we will plot.
# Calculate shap_values for all of val_X rather than a single row, to have more data for plot.
shap_values = explainer.shap_values(X_test)

# Make plot. Index of [1] is explained in text below.
shap.summary_plot(shap_values[1], X_test)

In [ ]:
# intrepetation:
# very little time at company negatively impacts quitting prediction, but medium values positively influence it. 
# spending a long time at the company doesn't much matter
# high salary scores negatively predict quitting
# seniority doesn't have much impact

In [ ]:
# calculate PDP comparing time at Co vs salary
# make plot.
shap.dependence_plot('salary', shap_values[1], X_test, interaction_index="time_at_Co")

In [ ]:
# vice versa
shap.dependence_plot('time_at_Co', shap_values[1], X_test, interaction_index="salary")

In [ ]:
# the best predictor seems to be time at the company. 
# would be itnerested in obtaining a 'satisfaction score' from employees. My bet is something is ticking them off